In [ ]:
import pygad
from sklearn.metrics import accuracy_score
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import joblib
import pyswarms as ps
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
import neat

In [36]:
data = pd.read_csv('csgo_filtred.csv')

y = data['bomb_planted']
X = data.drop(columns=['bomb_planted'], axis=1)
mapping = {
    'de_inferno': 1,
    'de_dust2': 2,
    'de_nuke': 3,
    'de_mirage': 4,
    'de_overpass': 5,
    'de_train': 6,
    'de_vertigo': 7,
    'unknown': 8,
    'de_cache': 9
}
X['map'] = X['map'].map(mapping)
scaler = StandardScaler()
X = scaler.fit_transform(X)
oversample = SMOTE()
transformed_X, tranformed_y = oversample.fit_resample(X, y)
X, y = transformed_X, tranformed_y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, train_size=0.005, random_state=42, stratify=y)

In [37]:
nb = joblib.load(r'models\naive_bayes')
dt = joblib.load(r'models\dt')

# PyGAD

In [38]:
def fitness_dt(ga_instance, solution, solution_idx):
    max_depth, min_split, min_leaf = map(int, solution)
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_split,
        min_samples_leaf=min_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

ga_dt = pygad.GA(
    num_generations=40,
    sol_per_pop=30,
    num_parents_mating=10,          
    num_genes=3,
    gene_space=[
        {'low': 1,  'high': 20},
        {'low': 2,  'high': 20},
        {'low': 1,  'high': 20}
    ],
    fitness_func=fitness_dt,
    parent_selection_type="sss",
    crossover_type="single_point",
    mutation_type="random"
)
ga_dt.run()
best_dt_solution, best_dt_fitness, _ = ga_dt.best_solution()
print("Лучшие параметры DT:", list(map(int, best_dt_solution)), "Accuracy:", best_dt_fitness)


def fitness_nb(ga_instance, solution, solution_idx):
    var_smoothing = 10 ** solution[0]
    model = GaussianNB(var_smoothing=var_smoothing)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

ga_nb = pygad.GA(
    num_generations=30,
    sol_per_pop=20,
    num_parents_mating=5,           
    num_genes=1,
    gene_space=[{'low': -12, 'high': -6}],
    fitness_func=fitness_nb,
    parent_selection_type="sss",
    crossover_type="single_point",
    mutation_type="random"
)

ga_nb.run()
best_nb_solution, best_nb_fitness, _ = ga_nb.best_solution()
best_var_smoothing = 10 ** best_nb_solution[0]
print("Лучший var_smoothing для NB:", best_var_smoothing, "Accuracy:", best_nb_fitness)


c:\Users\motyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\pygad\pygad.py:724: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(


Лучшие параметры DT: [16, 10, 4] Accuracy: 0.970873786407767


c:\Users\motyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\pygad\pygad.py:724: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(


Лучший var_smoothing для NB: 2.5945886040497872e-08 Accuracy: 0.9223300970873787


# PySwarms

In [39]:
def dt_objective(params):
    n_particles = params.shape[0]
    losses = np.zeros(n_particles)
    for i in range(n_particles):
        max_depth, min_split, min_leaf = map(int, params[i])
        model = DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_split=min_split,
            min_samples_leaf=min_leaf,
            random_state=42
        )
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        losses[i] = 1.0 - accuracy_score(y_test, preds)
    return losses

options_dt = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
bounds_dt = (np.array([1, 2, 1]), np.array([20, 20, 20]))

optimizer_dt = ps.single.GlobalBestPSO(
    n_particles=30,
    dimensions=3,
    options=options_dt,
    bounds=bounds_dt
)

best_cost_dt, best_pos_dt = optimizer_dt.optimize(
    dt_objective, iters=100, verbose=True
)

best_dt_params = list(map(int, best_pos_dt))
best_dt_acc = 1.0 - best_cost_dt
print("DT params:", best_dt_params, "accuracy:", best_dt_acc)


def nb_objective(params):
    n_particles = params.shape[0]
    losses = np.zeros(n_particles)
    for i in range(n_particles):
        log_vs = params[i][0]
        var_smoothing = 10 ** log_vs
        model = GaussianNB(var_smoothing=var_smoothing)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        losses[i] = 1.0 - accuracy_score(y_test, preds)
    return losses

options_nb = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
bounds_nb = (np.array([-12.0]), np.array([-6.0]))

optimizer_nb = ps.single.GlobalBestPSO(
    n_particles=20,
    dimensions=1,
    options=options_nb,
    bounds=bounds_nb
)

best_cost_nb, best_pos_nb = optimizer_nb.optimize(
    nb_objective, iters=80, verbose=True
)

best_log_vs = best_pos_nb[0]
best_nb_acc = 1.0 - best_cost_nb
best_var_smoothing = 10 ** best_log_vs
print("NB var_smoothing:", best_var_smoothing, "accuracy:", best_nb_acc)


2025-05-02 23:37:00,865 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.0291
2025-05-02 23:37:21,104 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.029126213592232997, best pos: [17.3727466   7.47608944  5.00632927]
2025-05-02 23:37:21,110 - pyswarms.single.global_best - INFO - Optimize for 80 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


DT params: [17, 7, 5] accuracy: 0.970873786407767


pyswarms.single.global_best: 100%|██████████|80/80, best_cost=0.0777
2025-05-02 23:37:24,105 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.07766990291262132, best pos: [-6.15701734]


NB var_smoothing: 6.965986979104352e-07 accuracy: 0.9223300970873787


# NEAT

In [51]:
small_data = pd.read_csv('cars_50_pairs_9_n_feature.csv')
big_data = pd.read_csv('cars_1430_pairs_8_n_feature.csv')

y_small = small_data['collision']
X_small = small_data.drop(columns=['collision'], axis=1)

y_big = big_data['collision']
X_big = big_data.drop(columns=['collision'], axis=1)

scaler = StandardScaler()
X_small = scaler.fit_transform(X_small)
X_big = scaler.fit_transform(X_big)

X_small_train, X_small_test, y_small_train, y_small_test = train_test_split(X_small, y_small, random_state=42, test_size=0.2, stratify=y_small)

X_big_train, X_big_test, y_big_train, y_big_test = train_test_split(X_big, y_big, random_state=42, test_size=0.2, stratify=y_big)

X_small_train.shape, X_big_train.shape

((40, 14), (1144, 12))

In [44]:
import os
import neat

config_path = "config-neat.ini"
config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path
)

pop = neat.Population(config)
pop.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
pop.add_reporter(stats)

def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        outputs = [net.activate(x) for x in X_train]
        y_pred = [int(np.argmax(o)) for o in outputs]
        genome.fitness = accuracy_score(y_train, y_pred)

winner = pop.run(eval_genomes, n=30)

best_net = neat.nn.FeedForwardNetwork.create(winner, config)
test_outs = [best_net.activate(x) for x in X_test]
y_pred_test = [int(np.argmax(o)) for o in test_outs]
test_acc = accuracy_score(y_test, y_pred_test)
print(f"\n=== NEAT Test Accuracy: {test_acc:.4f} ===")

RuntimeError: Missing configuration item: response_init_mean